### Metrics 1 - Fit, Eval and Save for Net3

In [11]:
import os, sys
import copy as copyroot
import pandas as pd
from IPython.display import display
from matplotlib import pyplot as plt
from fastai2.basics import *
from fastai2.vision.all import *
%load_ext autoreload
%autoreload 2

from module.mnist_helpers import build_df, eda_fig_1, build_dls
from module.mnist_models import FeatsNet
from module.mnist_metrics import metrics_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
dls_tl = build_dls(target='topleft')
dls_cr = build_dls(target='center')

In [13]:
x, y = dls_tl.one_batch()
x.shape

torch.Size([64, 1, 28, 28])

In [14]:
def build_net3():
    net3 = nn.Sequential(
                nn.Flatten(),
                nn.Linear(in_features=784, out_features=28),
                nn.Linear(in_features=28, out_features=2, bias=False),
                SigmoidRange(-1., 1),
                )
    return net3


In [15]:
net3_tl = Learner(dls_tl, build_net3(), loss_func=MSELossFlat(), )
net3_tl.fit(10)

epoch,train_loss,valid_loss,time
0,0.102103,0.036844,00:01
1,0.070461,0.046114,00:01
2,0.057702,0.032345,00:01
3,0.047579,0.025123,00:01
4,0.040680,0.025922,00:01
5,0.035446,0.022276,00:01
6,0.031565,0.023018,00:01
7,0.028455,0.020932,00:01
8,0.026049,0.021573,00:01
9,0.023888,0.020588,00:01


In [16]:
net3_cr = Learner(dls_cr, build_net3(), loss_func=MSELossFlat(), )
net3_cr.fit(10)

epoch,train_loss,valid_loss,time
0,0.006305,0.005020,00:01
1,0.004953,0.003183,00:01
2,0.004027,0.002701,00:01
3,0.003448,0.002816,00:01
4,0.003042,0.002625,00:01
5,0.002760,0.002681,00:01
6,0.002524,0.002429,00:01
7,0.002331,0.002362,00:01
8,0.002162,0.002254,00:01
9,0.002001,0.002321,00:01


In [17]:
df_tl = metrics_df(net3_tl, 'Net3', '10 fit epochs', 'topleft')
df_tl

,model,details,target,split,mse,mae,r2,dist_avg,dist_r2,sqdist_avg,sqdist_r2
0,Net3,10 fit epochs,topleft,valid,0.020588,0.103328,0.170152,0.166890,0.227500,0.041177,0.355698
1,Net3,10 fit epochs,topleft,train,0.014115,0.086962,0.388219,0.138814,0.409787,0.028231,0.628363


In [18]:
df_cr = metrics_df(net3_cr, 'Net3', '10 fit epochs', 'center')
df_cr

,model,details,target,split,mse,mae,r2,dist_avg,dist_r2,sqdist_avg,sqdist_r2
0,Net3,10 fit epochs,center,valid,0.002321,0.035860,0.529265,0.057386,0.430105,0.004642,0.625702
1,Net3,10 fit epochs,center,train,0.001251,0.026886,0.780596,0.042757,0.590545,0.002502,0.810111


In [19]:
df = pd.concat((df_cr, df_tl), axis=0)
df

,model,details,target,split,mse,mae,r2,dist_avg,dist_r2,sqdist_avg,sqdist_r2
0,Net3,10 fit epochs,center,valid,0.002321,0.035860,0.529265,0.057386,0.430105,0.004642,0.625702
1,Net3,10 fit epochs,center,train,0.001251,0.026886,0.780596,0.042757,0.590545,0.002502,0.810111
0,Net3,10 fit epochs,topleft,valid,0.020588,0.103328,0.170152,0.166890,0.227500,0.041177,0.355698
1,Net3,10 fit epochs,topleft,train,0.014115,0.086962,0.388219,0.138814,0.409787,0.028231,0.628363


In [20]:
dir_fn = 'assets/metrics-dfs/'
df.to_csv(dir_fn + 'metrics1-df-net3.csv', index=False)